# LCEL进阶

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [12]:
from langchain.globals import set_debug
set_debug(False) 

## 绑定常量参数
有时，我们希望调用`Runnable`序列中的`Runnable`，其常量参数不属于序列中前面`Runnable`的输出，也不属于用户输入。我们可以使用[Runnable.bind()](https://api.python.langchain.com/en/latest/schema.runnable/langchain.schema.runnable.base.Runnable.html#langchain.schema.runnable.base.Runnable.bind)来传递这些参数。


首先构建一个简单的`prompt + model`链条，以交互的方式引导用户提供一个等式，然后根据该等式进行求解。代码中使用了 `ChatPromptTemplate` 来定义交互模板，`ChatOpenAI` 模型进行对话，然后使用 `StrOutputParser` 解析输出结果（把响应作为字符串返回）。


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n"),
    ("human", "{equation_statement}")
])
model = ChatOpenAI(temperature=0)
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))


[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "x raised to the third plus seven equals 12"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<equation_statement>] Entering Chain run with input:
{
  "input": "x raised to the third plus seven equals 12"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<equation_statement> > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "x raised to the third plus seven equals 12"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<equation_statement> > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "x raised to the third plus seven equals 12"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<equation_statement>] s] Exiting Chain run with output:
{
  "equation_statement": "x raised to the third plus seven equals 12"
}
[chain/start] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Enter

但有时候，我们想要在这个序列中的某一步向模型传递一些额外的参数，而这些参数并不是前一步输出的结果，也不是用户输入的一部分。这时候就可以使用 `RunnablePassthrough()` 来将这些常量参数传递给模型。

我们使用`model.bind(stop="SOLUTION")`，将一个名为 `stop` 的参数绑定到 `model` 可运行对象上，并传递值 "SOLUTION"。这样，模型在生成响应时，看到"SOLUTION"后就会停止响应，即求解等式后停止。



In [13]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))


EQUATION: x^3 + 7 = 12




因此，通过`bind`可以在不改变`Prompt`的情况下，在序列中的不同步骤中灵活地传递参数，修改模型的运行方式。这样的设计使得处理复杂的操作序列更加简洁和灵活。

## 附加OpenAI函数
绑定的一个特别有用的应用是将OpenAI函数附加到兼容的OpenAI模型上

In [14]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

In [19]:
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4", temperature=0).bind(
    function_call={"name": "solver"}, functions=[function]
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
res = runnable.invoke("x raised to the third plus seven equals 12")
res.additional_kwargs

{'function_call': {'arguments': '{\n"equation": "x^3 + 7 = 12",\n"solution": "x = ∛5"\n}',
  'name': 'solver'}}

## 附加OpenAI工具


In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

model = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(tools=tools)
res = model.invoke("What's the weather in SF, NYC and LA?")


In [18]:
res.additional_kwargs

{'tool_calls': [{'id': 'call_LQMw4NLOJTLfYNhuVEPmmDr5',
   'function': {'arguments': '{"location": "San Francisco, CA", "unit": "celsius"}',
    'name': 'get_current_weather'},
   'type': 'function'},
  {'id': 'call_t1CIVo0HlGu12U4HdIEgCxtN',
   'function': {'arguments': '{"location": "New York, NY", "unit": "celsius"}',
    'name': 'get_current_weather'},
   'type': 'function'},
  {'id': 'call_LbO65il3Ft0512X4wke7nHmA',
   'function': {'arguments': '{"location": "Los Angeles, CA", "unit": "celsius"}',
    'name': 'get_current_weather'},
   'type': 'function'}]}

## RunnableBinding 函数

In [21]:
from langchain_core.runnables import RunnableBinding
model = ChatOpenAI(temperature=0)
runnable_binding = RunnableBinding(
    bound=model,
    kwargs={'stop': ['-']} # <-- Note the additional kwargs
)
runnable_binding.invoke('Say "Parrot-MAGIC"') # Should return `Parrot`

AIMessage(content='Parrot', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 14, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-de6af5ca-be8e-4f51-8346-555a666ba7ad-0')

## 配置和自定义chain
